# Generic system-near OpenMath serialization for Python: a demo

## Abstract
This notebook demonstrates the OpenMath serialization for Python implemented
in the [convert-pickle branch](https://github.com/OpenMath/py-openmath/tree/convert-pickle) of py-openmath.

With it, any Python object that can be *pickled* (the standard serialization mechanism of Python) can be exported into an OpenMath expression tree that can be reevaluated back to the original object.

Caveats:
- The expression tree is very much system near: it describes how the object can be reconstructed from basic Python objects by application of standard Python functions.
- This currently only works with Python 2 (and therefore Sage)
- Some constructs are not yet implemented
- Although there is nothing specific to Sage per se, several of the examples below are Sage specific

## Try it

### Simplest: run this notebook on [Binder@EGI](https://binderhub.fedcloud-tf.fedcloud.eu/v2/gh/OpenDreamKit/MitM-Sage/master?filepath=sage/openmath_pickle_demo.ipynb)

### Install the required software

    git clone https://github.com/OpenMath/py-openmath.git -b convert-pickle 
    cd py-openmath
    pip install -e .     # Assuming pip is Python's 2 pip

Variant for use in SageMath:

    sage -pip install -e .                                 # Sage

Download and open the notebook:

    jupyter notebook openmath_pickle_demo.ipynb            # Python
    sage -notebook jupyter openmath_pickle_demo.ipynb      # Sage
    
The `-e` means that `py-openmath` has been installed in editable mode; so you can edit the sources for customization.

## A first OpenMath serialization and deserialization: Sage integers

In [1]:
from openmath.convert_pickle import to_openmath, to_python, test_openmath
from openmath.encoder import encode_xml
from lxml import etree

A Sage integer is serialized as an application of the function `sage.rings.integer.make_integer` applied on a (base 32?) string representation:

In [2]:
o = to_openmath(42); o

OMApplication(elem=OMSymbol(name='make_integer', cd='sage.rings.integer', id=None, cdbase='http://python.org'), arguments=[OMBytes(bytes='1a', id=None)], id=None, cdbase=None)

Pretty printing the OpenMath object

In [3]:
print(o)

OMApplication(
  elem=OMSymbol(name='make_integer', cd='sage.rings.integer', cdbase='http://python.org'),
  arguments=[OMBytes(bytes='1a')])


Pretty printing the XML

In [4]:
print(etree.tostring(encode_xml(o), pretty_print=True))

<OMA xmlns="http://www.openmath.org/OpenMath">
  <OMS name="make_integer" cd="sage.rings.integer"/>
  <OMB>MWE=</OMB>
</OMA>



For deserialization, we can use the plain openmath deserialization from [py-openmath](https://github.com/OpenMath/py-openmath):

In [5]:
to_python(o)

42

This really is a Sage integer:

In [6]:
type(_)

<type 'sage.rings.integer.Integer'>

Note that the above is a **system-near OpenMath serialization**. Deserializing to another system will further require **alignments** with its own constructors (a Rosetta stone). This will be the job of the Math-in-the-Middle engine.

This utility checks that serialization and deserialization gives an equal object of the same type:

In [7]:
test_openmath(1)

## Serialization of basic Python types

For basic Python types, we get the standard OpenMath serialization:

In [8]:
to_openmath(42r)

OMInteger(integer=42, id=None)

In [9]:
to_openmath(None)

OMSymbol(name='None', cd='__builtin__', id=None, cdbase='http://python.org')

In [10]:
l = [1r,2r,3r]
print(to_openmath(l))

OMApplication(
  elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
  arguments=[
    OMInteger(integer=1),
    OMInteger(integer=2),
    OMInteger(integer=3)])


In [11]:
test_openmath(l)

In [12]:
d = {1r: 'foo', 'bar': 2r}
print(to_openmath(d))

OMApplication(
  elem=OMSymbol(name='dict', cd='__builtin__', cdbase='http://python.org'),
  arguments=[OMApplication(
    elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
    arguments=[
      OMApplication(
        elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
        arguments=[
          OMInteger(integer=1),
          OMBytes(bytes='foo')]),
      OMApplication(
        elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
        arguments=[
          OMBytes(bytes='bar'),
          OMInteger(integer=2)])])])


In [13]:
for obj in [None, int(42), 'coucou', l, d]:
    test_openmath(obj)

In Python 2, non unicode strings are currently rendered as OMBytes

In [14]:
to_openmath('coucou')

OMBytes(bytes='coucou', id=None)

## Non trivial examples: Permutations groups and elements thereof

In [15]:
G = DihedralGroup(3)

In [16]:
print(to_openmath(G))

OMApplication(
  elem=OMSymbol(name='unreduce', cd='sage.structure.unique_representation', cdbase='http://python.org'),
  arguments=[
    OMSymbol(name='DihedralGroup', cd='sage.groups.perm_gps.permgroup_named', cdbase='http://python.org'),
    OMApplication(
      elem=OMSymbol(name='tuple_from_sequence', cd='openmath.convert_pickle', cdbase='http://python.org'),
      arguments=[OMApplication(
        elem=OMSymbol(name='make_integer', cd='sage.rings.integer', cdbase='http://python.org'),
        arguments=[OMBytes(bytes='3')])]),
    OMApplication(
      elem=OMSymbol(name='dict', cd='__builtin__', cdbase='http://python.org'),
      arguments=[OMApplication(
        elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
        arguments=[])])])


In [17]:
to_python(_)

'coucou'

In [18]:
g = G.an_element(); g

(1,2,3)

In [19]:
o = to_openmath(g)
print(o)

OMApplication(
  elem=OMSymbol(name='make_permgroup_element_v2', cd='sage.groups.perm_gps.permgroup_element', cdbase='http://python.org'),
  arguments=[
    OMApplication(
      elem=OMSymbol(name='unreduce', cd='sage.structure.unique_representation', cdbase='http://python.org'),
      arguments=[
        OMSymbol(name='DihedralGroup', cd='sage.groups.perm_gps.permgroup_named', cdbase='http://python.org'),
        OMApplication(
          elem=OMSymbol(name='tuple_from_sequence', cd='openmath.convert_pickle', cdbase='http://python.org'),
          arguments=[OMApplication(
            elem=OMSymbol(name='make_integer', cd='sage.rings.integer', cdbase='http://python.org'),
            arguments=[OMBytes(bytes='3')])]),
        OMApplication(
          elem=OMSymbol(name='dict', cd='__builtin__', cdbase='http://python.org'),
          arguments=[OMApplication(
            elem=OMSymbol(name='list', cd='list1', cdbase='http://www.openmath.org/cd'),
            arguments=[])])]),
    OMApp

In [20]:
to_python(o)

(1,2,3)